In [ ]:
from pybeam.datamodels import nodes_from_csv, elements_from_csv
from pybeam.assembly import assemble_system_matrices, reindex_dof
from pybeam.simulation import simulate
from pybeam.modal_parameters import get_modal_parameters
from pybeam._utilities import pprint_array
from pybeam.plotting import plot_structure, plot_modeshape, set_style, plot_deformations
from pybeam.fatigue import displacements_to_stresses

%matplotlib qt5
set_style()
import numpy as np

import logging

logging.getLogger("pybeam").setLevel(logging.INFO)

### Initialize system matrices

Assemble the system matrices.

The damping matrix is defined by Rayleigh's damping model. Change coefficients as needed.

In [ ]:
nodes = nodes_from_csv("data/frame/nodes.csv")
elements = elements_from_csv("data/frame/elements.csv", nodes)

stiffness, mass = assemble_system_matrices(elements)
damping = 0.01 * mass + 0.001 * stiffness
# damping = None

elements = reindex_dof(elements)

In [ ]:
loads = np.zeros((stiffness.shape[0]))
loads[10] = 50

displacements = np.linalg.inv(stiffness).dot(loads)

In [ ]:
plot_deformations(elements, displacements)

### Get stresses in an element

`displacements_to_stresses` takes the global displacement vector (`displacements`), one element (`element`) and the distance from the neutral axis of the element to the outermost fibre (`z`).

The result is a 2-item list, where each item is a 2-item tuple. Each item in the list corresponds to a node, so `stress[0]` are the stresses of the start node and `stress[1]` are the stresses of the end node.

The tuples are the pair of (normal stress, bending stress). Therefore, `stress[0][0]` is the normal stress in the start node and `stress[1][1]` is the bending stress in the end node.

In [ ]:
stress = displacements_to_stresses(displacements, elements[0], z=0.01)

print(f"Normal stress at start node is {stress[0][0]}")
print(f"Normal stress at end node is {stress[1][0]}")
print(f"Bending stress at start node is {stress[0][1]}")
print(f"Bending stress at end node is {stress[1][1]}")